In [12]:
#test the litter set
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.cluster import KMeans
from tensorflow.keras.datasets import mnist

from time import time
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input,Layer, InputSpec
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import callbacks
from tensorflow.keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import metrics

print(__import__('sys').version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)
# 10 clusters
n_clusters = len(np.unique(y))
# Runs in parallel 4 CPUs
kmeans = KMeans(n_clusters=n_clusters, n_init=20, n_jobs=4)
# Train K-Means.
y_pred_kmeans = kmeans.fit_predict(x)
# Evaluate the K-Means clustering accuracy.


In [ ]:
print(accuracy_score(y, y_pred_kmeans))
metrics.acc(y, y_pred_kmeans)

In [2]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')


In [66]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)
x.shape

(60000, 28, 28)
(10000, 28, 28)


(70000, 784)

In [64]:
n_clusters = len(np.unique(y))
print(n_clusters)
x.shape

10


(70000, 784)

In [5]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = 300
batch_size = 256
save_dir = './results'

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
autoencoder, encoder = autoencoder(dims, init=init)

In [16]:
#from tensorflow.keras.utils import plot_model
#plot_model(autoencoder, to_file='autoencoder.png', show_shapes=True)
#from IPython.display import Image
#Image(filename='autoencoder.png') 

In [17]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)
autoencoder.save_weights(save_dir + '/ae_weights.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
70000/70000 [==============================] - 20s 286us/sample - loss: 0.0646
Epoch 2/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0436
Epoch 3/300
70000/70000 [==============================] - 1s 17us/sample - loss: 0.0318
Epoch 4/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0266
Epoch 5/300
70000/70000 [==============================] - 1s 17us/sample - loss: 0.0239
Epoch 6/300
70000/70000 [==============================] - 1s 17us/sample - loss: 0.0223
Epoch 7/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0211
Epoch 8/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0202
Epoch 9/300
70000/70000 [==============================] - 1s 18us/sample - loss: 0.0195
Epoch 10/300
70000/70000 [==============================] - 2s 22us/sample - loss: 0.0189
Epoch 11/30

70000/70000 [==============================] - 1s 15us/sample - loss: 0.0109
Epoch 87/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0109
Epoch 88/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0109
Epoch 89/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0108
Epoch 90/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0108
Epoch 91/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0108
Epoch 92/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0108
Epoch 93/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0107
Epoch 94/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0107
Epoch 95/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0107
Epoch 96/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0106
Epoch 97/300
70000/7000

70000/70000 [==============================] - 1s 17us/sample - loss: 0.0092
Epoch 177/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0092
Epoch 178/300
70000/70000 [==============================] - 1s 14us/sample - loss: 0.0092
Epoch 179/300
70000/70000 [==============================] - 1s 14us/sample - loss: 0.0092
Epoch 180/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0092
Epoch 181/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0092
Epoch 182/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0092
Epoch 183/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0092
Epoch 184/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0091
Epoch 185/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0091
Epoch 186/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0091
Epoch 187/300

70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 267/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 268/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 269/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 270/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 271/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 272/300
70000/70000 [==============================] - 1s 15us/sample - loss: 0.0084
Epoch 273/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0084
Epoch 274/300
70000/70000 [==============================] - 1s 17us/sample - loss: 0.0084
Epoch 275/300
70000/70000 [==============================] - 1s 17us/sample - loss: 0.0084
Epoch 276/300
70000/70000 [==============================] - 1s 16us/sample - loss: 0.0084
Epoch 277/300

OSError: Unable to create file (unable to open file: name = './results/ae_weights.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [19]:
autoencoder.save_weights(save_dir + '/ae_weights.h5')

In [ ]:
autoencoder.load_weights(save_dir + '/ae_weights.h5')

In [41]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(name="clustering", shape = (int(self.n_clusters), int(input_dim)), initializer='glorot_uniform',trainable=True)
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [47]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)

In [48]:
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

In [49]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))

In [50]:
y_pred_last = np.copy(y_pred)

In [51]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

In [52]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [53]:
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(x.shape[0])

In [54]:
tol = 0.001 # tolerance threshold to stop training

In [55]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(metrics.acc(y, y_pred), 5)
            nmi = np.round(metrics.nmi(y, y_pred), 5)
            ari = np.round(metrics.ari(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/DEC_model_final.h5')

/home/king/anaconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


Iter 0: acc = 0.91256, nmi = 0.82029, ari = 0.81521  ; loss= 0
Iter 140: acc = 0.92089, nmi = 0.83093, ari = 0.83332  ; loss= 0.04619
Iter 280: acc = 0.93281, nmi = 0.85022, ari = 0.85777  ; loss= 0.15459
Iter 420: acc = 0.94059, nmi = 0.86294, ari = 0.87380  ; loss= 0.2047
Iter 560: acc = 0.94543, nmi = 0.87193, ari = 0.88383  ; loss= 0.21731
Iter 700: acc = 0.94887, nmi = 0.87776, ari = 0.89094  ; loss= 0.21237
Iter 840: acc = 0.95151, nmi = 0.88263, ari = 0.89639  ; loss= 0.20751
Iter 980: acc = 0.95300, nmi = 0.88542, ari = 0.89950  ; loss= 0.20571
Iter 1120: acc = 0.95427, nmi = 0.88787, ari = 0.90218  ; loss= 0.19575
Iter 1260: acc = 0.95507, nmi = 0.88937, ari = 0.90386  ; loss= 0.19596
Iter 1400: acc = 0.95580, nmi = 0.89090, ari = 0.90536  ; loss= 0.19036
Iter 1540: acc = 0.95640, nmi = 0.89227, ari = 0.90663  ; loss= 0.17978
Iter 1680: acc = 0.95674, nmi = 0.89285, ari = 0.90733  ; loss= 0.17421
Iter 1820: acc = 0.95733, nmi = 0.89427, ari = 0.90855  ; loss= 0.18425
Iter 1960

In [56]:
model.load_weights(save_dir + '/DEC_model_final.h5')

In [57]:
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    acc = np.round(metrics.acc(y, y_pred), 5)
    nmi = np.round(metrics.nmi(y, y_pred), 5)
    ari = np.round(metrics.ari(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('Acc = %.5f, nmi = %.5f, ari = %.5f' % (acc, nmi, ari), ' ; loss=', loss)

Acc = 0.95873, nmi = 0.89781, ari = 0.91153  ; loss= 0.10534


In [58]:
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt
sns.set(font_scale=3)
confusion_matrix = sklearn.metrics.confusion_matrix(y, y_pred)

plt.figure(figsize=(16, 14))
sns.heatmap(confusion_matrix, annot=True, fmt="d", annot_kws={"size": 20});
plt.title("Confusion matrix", fontsize=30)
plt.ylabel('True label', fontsize=25)
plt.xlabel('Clustering label', fontsize=25)
plt.show()

<Figure size 1600x1400 with 2 Axes>

In [59]:
from sklearn.utils.linear_assignment_ import linear_assignment

y_true = y.astype(np.int64)
D = max(y_pred.max(), y_true.max()) + 1
w = np.zeros((D, D), dtype=np.int64)
# Confusion matrix.
for i in range(y_pred.size):
    w[y_pred[i], y_true[i]] += 1
ind = linear_assignment(-w)

sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

0.9587285714285714

In [60]:
w

array([[6795,    2,   32,    3,    4,   28,   31,    8,   18,   28],
       [   1,   29,   52,   53,    8,    3,    0, 6978,    9,  127],
       [  47,   45,  107,   56,    9,   18,   11,    8, 6434,   55],
       [   6,   91, 6674,  113,   16,   19,    5,   97,   41,    4],
       [   5,   13,   51,    1, 6633,    8,   13,   45,   18,   91],
       [   0, 7645,    7,    1,    4,    3,   10,   28,   30,   10],
       [   4,   35,   43, 6820,    0,   91,    0,    8,   98,  124],
       [   6,    2,    2,   76,    0, 6073,  242,    3,   74,   20],
       [  28,    4,   15,    2,   34,   47, 6564,    0,   12,    4],
       [  11,   11,    7,   16,  116,   23,    0,  118,   91, 6495]])

In [61]:
ind

array([[0, 0],
       [1, 7],
       [2, 8],
       [3, 2],
       [4, 4],
       [5, 1],
       [6, 3],
       [7, 5],
       [8, 6],
       [9, 9]])

In [62]:
w.argmax(1)

array([0, 7, 8, 2, 4, 1, 3, 5, 6, 9])